# Python爬虫教程

课程来源：[Python超强爬虫8天速成（完整版）爬取各种网站数据实战案例](https://www.bilibili.com/video/BV1ha4y1H7sx?p=6&spm_id_from=pageDriver)

## 使用场景

通用爬虫：抓取系统的组成部分，抓取一张页面数据

聚焦爬虫：在通用的基础上，抓取特定内容

增量式爬虫：检测网站中数据更新

## http协议
服务器与客户端进行数据交互的一种形式

常用请求头信息

- User-Agent：请求载体的身份标识

- Connection：请求完毕后，是断开连接还是保持

常用响应头协议
- Content-Type：服务器响应客户端的数据类型

## https协议

- 安全的超文本协议

加密方式

- 对称密钥加密

- 非对称密钥加密

- 证书密钥加密

---

## requests模块

作用：模拟浏览器发送请求

如何使用：

- 指定url

- 发起请求

- 获取响应数据

- 持久化存储

In [8]:
#需求：爬取搜狗首页
import requests
#指定url
url = 'https://www.sogou.com/'
#发起请求
#返回响应对象
response = requests.get(url = url)
#获取响应数据
#获取字符串形式的响应数据
page_text = response.text
print(page_text)
#持久化存储
with open('./sougou.html', 'w', encoding ='utf-8') as fp:
    fp.write(page_text)
    print('爬取结束！')

<!DOCTYPE html><html lang="cn"><head><meta name="viewport" content="width=device-width,minimum-scale=1,maximum-scale=1,user-scalable=no"><script>window._speedMark = new Date();  window.lead_ip = '120.235.33.120';
    window.now = 1654683051613;</script><script type="text/javascript">/*file=static/js/resourceErrorReport.js*/!function(a){var n=(new Date).getTime(),r=a.location.protocol;function c(e,t){var o=(new Date).getTime()-n;(new Image).src=["//pb.sogou.com/pv.gif?uigs_productid=wapapp&type=resource-error&stype=",e,"&timestamp=",o,"&protocol=",r,"&host=",encodeURIComponent(a.location.host),"&path=",encodeURIComponent(a.location.pathname),"&resource=",encodeURIComponent(t)].join("")}function e(e){if((e=e||a.event)&&"error"===e.type){var t=e.srcElement?e.srcElement:e.target;if(t){var o,n,r=t.tagName;"LINK"===r?(n="css",(o=t.getAttribute("href"))&&o.match(/\.css($|\?)/)&&c(n,o)):"SCRIPT"===r&&(n="js",(o=t.getAttribute("src"))&&o.match(/\.js($|\?)/)&&c(n,o))}}}r&&(r=r.substring(0,r.leng

## 案例

- 指定搜索结果页面

- 破解百度翻译

- 豆瓣电影分类排行榜

- 肯德基餐厅查询

- 国家药监局相关数据

## 指定搜索结果页面

### User-Agent伪装

UA检测：

门户网站的服务器检测对应的请求对应载体身份标识，
若检测到载体的身份标识是某一浏览器，则为正常请求，
若检测载体的身份标识为非浏览器，则异常，可能拒绝请求。

查看方式：

`Shift + Ctrl + I`打开浏览器开发者工具，点击网络，选中网站，点击标头，找到User-Agent

In [ ]:
#指定搜索结果页面
import requests
#UA伪装
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
url = 'https://www.sogou.com/web'
#处理url携带的参数：封装到字典中
kw = input('enter a word:')
param = {
    'query' : kw
}
#对指定url发起请求是携带参数的，且请求过程中处理了参数
response = requests.get(url = url, params = param, headers = headers)
page_text = response.text
filename = kw + '.html'
with open(filename, 'w', encoding = 'utf-8') as fp:
    fp.write(page_text)
print(filename, '保存成功！')

## 破解百度翻译

刷新页面的一部分

### 相关知识

- post请求（携带了参数）

- 响应数据是一组

### 操作

1. `Shift + Ctrl + I`打开开发者工具，点击网络，点击Fetch/XHR

2. 点击名称，点击负载，找到显示关键词的组

3. 返回标头，复制url

4. 在标头中找到Content-Type，显示json则为json方法

In [ ]:
#破解百度翻译
import requests
import json
post_url = 'https://fanyi.baidu.com/sug'
#UA伪装
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
#post请求参数处理（与get相同）
word = input('enter a word:')
data = {
    'kw' : word
}
#请求发送
response = requests.post(url = post_url, data = data, headers = headers)
#获取相应数据:json方法返回的是object(响应数据为json类型，才能使用)
dic_obj = response.json()
#持久化存储
filename = word + '.json'
fp = open(filename, 'w', encoding = 'utf-8')
json.dump(dic_obj, fp = fp, ensure_ascii = False)
print('over!')

## 豆瓣电影分类排行榜

下拉页面会进行部分刷新，如增加电影数量没滚动条位置变化

In [ ]:
#豆瓣电影分类排行榜
import requests
import json
url = 'https://movie.douban.com/j/chart/top_list'
param = {
    'type' : '24',
    'interval_id' : '100:90',
    'action' : '',
    'start' : '0', #从库中第几部电影开始取
    'limit' : '20', #单次请求取出的个数
}
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
response = requests.get(url = url, params = param, headers = headers)
list_data = response.json()
fp = open('./douban.json', 'w', encoding = 'utf-8')
json.dump(list_data, fp = fp, ensure_ascii = False)
print('over!')

## 作业：肯德基餐厅查询

重点：

1. 确定网址的url

2. 确定post请求还是get请求

3. 确定响应数据是text还是json

In [ ]:
#作业：肯德基餐厅查询
import requests
url = 'https://www.kfc.com.cn/kfccda/ashx/GetStoreList.ashx?op=keyword'
word = input('enter a city:')
data = {
    'cname' : '',
    'pid' : '',
    'keyword' : word,
    'pageIndex' : '1',
    'pageSize' : '10',
}
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
response = requests.post(url = url, data = data, headers = headers)
page_text = response.text
filename = word + '.html'
with open(filename, 'w', encoding = 'utf-8') as fp:
    fp.write(page_text)
print(filename, '保存成功！')

## 国家药监局基于化妆品生产许可证相关数据

网址：[化妆品生产许可信息管理平台](http://scxk.nmpa.gov.cn:81/xk/)

需求：需要爬取企业的具体信息，非主业列表

并不一定能直接从地址栏中请求得到

可能是**动态加载**出来的数据

对应企业的数据是通过ajax动态请求得到的

**<font color=red>未解决</font>**

In [ ]:
#国家药监局基于化妆品生产许可证相关数据
import requests
url = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList'
data = {
    'on': 'true',
    'page': '1',
    'pageSize': '15',
    'productName': '',
    'conditionType': '1',
    'applyname': '',
    'applysn': '',
}
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
id_list = []
json_ids = requests.post(url = url, headers = headers, data = data)
for dic in json_ids['list']:
    id_list.append(dic['ID'])

---

## 数据解析

聚焦爬虫：爬取页面指定内容

分类：正则、bs4、**xpath**

原理：

- 解析局部文本内容在标签之间或标签对应的属性中存储

- 指定标签进行定位

- 标签或标签属性中存储的数据值进行提取（解析）

流程：

1. 指定url

2. 发起请求

3. 获取相应数据

4. 数据解析

5. 持久化存储

In [ ]:
#爬取图片数据
import requests
url = 'http://mari.hzau.edu.cn/__local/B/C9/4C/40EE6E9C872EBEF7B0E864D8540_34AE630B_15EF3.jpg'
#content返回二进制图像数据
#text字符串，content二进制，json对象
img_data = requests.get(url = url).content
with open('./image.jpg', 'wb') as fp:
    fp.write(img_data)

In [ ]:
#爬取所有图片数据
import requests
import re
import os
#创建文件夹
if not os.path.exists('./marilibs'):
    os.mkdir('./marilibs')
#爬取页面信息
url = 'http://mari.hzau.edu.cn/info/1003/3711.htm'
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
page_text = requests.post(url = url, headers = headers).text
#获取图像数据
#正则表达式
ex = '<p style="text-align: center"><img src="(.*?)" width'
img_src_list = re.findall(ex, page_text, re.S)
print(img_src_list)
#拼接图片地址
for src in img_src_list:
    src = 'http://mari.hzau.edu.cn' + src
    img_data = requests.get(url = src, headers = headers).content
#存储图片
    img_name = src.split('/')[-1]
    imgPath = './marilibs/' + img_name
    with open(imgPath, 'wb') as fp:
        fp.write(img_data)
        print(img_name, '下载成功！')

## 分页爬取

url当中包含通用，也包含页码参数，通过循环将页码拼接到通用url当中进行循环

网页来源：[【壁纸】抱图吱声 壁纸头像各种类型](https://jump2.bdimg.com/p/4378860308)

In [ ]:
#爬取贴吧壁纸图片数据
import requests
import re
import os
#创建文件夹
if not os.path.exists('./wallpapers'):
    os.mkdir('./wallpapers')
#UA伪装
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
#设置通用url模板
url = 'https://jump2.bdimg.com/p/4378860308?pn=%d'
pageNum = 1
for pageNum in range(1, 5):
    new_url = format(url%pageNum) 
    page_text = requests.post(url = new_url, headers = headers).text
#获取图像数据
#正则表达式
    ex = '<img class="BDE_Image" src="(.*?)" size'
    img_src_list = re.findall(ex, page_text, re.S)
    print(img_src_list)
#拼接图片地址
    for src in img_src_list:
        src = src
        img_data = requests.get(url = src, headers = headers).content
#存储图片
        img_name = src.split('/')[-1]
        imgPath = './wallpapers/' + img_name
        with open(imgPath, 'wb') as fp:
            fp.write(img_data)
            print(img_name, '下载成功！')

## bs4解析

Python独有的解析方式

原理：
- 实例化一个BeautifulSoup对象，并将页面源码加载到该对象中

- 通过调用BeautifulSoup对象中的相关属性或方法对标签进行定位和数据提取

安装：`bs4`，`lxml`

对象的实例化：

- 将本地html文档数据加载到对象当中
    
```python
fp = open('./test.html', 'r', encoding = 'utf-8')
soup = BeautifulSoup(fp, 'lxml')
```

- 从网上获取源码加载到该对象中
    
```python
page_text = response.text    
soup = BeautifulSoup(page_text, 'lxml')
```

提供的用于数据解析的方法和属性：

- `soup.tagName`返回的是文档中第一个出现的tagName标签

- `soup.find`

    - 等同于soup.tagName
    
    - 属性定位：`soup.find('div', class_/id/attr = 'song')`
    
- `soup.find_all(tagName)`返回符合要求的所有标签

- `soup.slelct`

    - `select('某种选择器(id, class, 标签...选择器)')`，返回的是一个列表
    
    - `soup.select('.tang > ul > li > a')[0])`，层级选择器，大于号表示一个层级
    
    -  `print(soup.select('.tang > ul a'))`，空格表示的是多个层级

获取标签之间文本数据：

- `soup.a.text/string/get_text()`

- `text和get_text()`可以获取某一标签下所有的文本内容

- `string`只可以获取该标签下的直系文本

获取标签中属性值：

- `soup.a['href']`

In [9]:
#将本地html文档数据加载到对象当中
from bs4 import BeautifulSoup
fp = open('./test.html', 'r', encoding = 'utf-8')
soup = BeautifulSoup(fp, 'lxml')
#print(soup)
#print(soup.a) #soup.tagName返回的是html中第一个出现的a标签
#print(soup.div)
#print(soup.find('div')) #等同于soup.div
#print(soup.find('div', class_ = 'song')) #属性定位
#print(soup.find_all('a'))
#print(soup.select('.tang'))
#print(soup.select('.tang > ul > li > a')[0])
#print(soup.select('.tang > ul a')[0].get_text())
#print(soup.select('.tang > ul a')[0]['href'])

http://www.baidu.com


In [23]:
#爬取三国演义小说所有章节的标题和内容
import requests
from bs4 import BeautifulSoup
#爬取首页页面数据
url = 'https://www.shicimingju.com/book/sanguoyanyi.html'
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
page_text = requests.get(url = url, headers = headers).content
#解析章节标题与详情页
soup = BeautifulSoup(page_text, 'lxml')
li_list = soup.select('.book-mulu > ul > li')
fp = open('./sanguo.txt', 'w', encoding = 'utf-8')
for li in li_list:
    title = li.a.string
    detail_url = 'https://www.shicimingju.com' + li.a['href']
    #对详情页发起请求
    detail_page_text = requests.get(url = detail_url, headers = headers).content
    #解析详情页内容
    detail_soup = BeautifulSoup(detail_page_text, 'lxml')
    div_tag = detail_soup.find('div', class_ = 'chapter_content')
    content = div_tag.text
    fp.write(title + ':' +content + '\n')
    print(title, '爬取成功！')

第一回·宴桃园豪杰三结义  斩黄巾英雄首立功 爬取成功！
第二回·张翼德怒鞭督邮    何国舅谋诛宦竖 爬取成功！
第三回·议温明董卓叱丁原  馈金珠李肃说吕布 爬取成功！
第四回·废汉帝陈留践位    谋董贼孟德献刀 爬取成功！
第五回·发矫诏诸镇应曹公  破关兵三英战吕布 爬取成功！
第六回·焚金阙董卓行凶    匿玉玺孙坚背约 爬取成功！
第七回·袁绍磐河战公孙    孙坚跨江击刘表 爬取成功！
第八回·王司徒巧使连环计  董太师大闹凤仪亭 爬取成功！
第九回·除暴凶吕布助司徒  犯长安李傕听贾诩 爬取成功！
第一十回·勤王室马腾举义    报父仇曹操兴师 爬取成功！
第十一回·刘皇叔北海救孔融  吕温侯濮阳破曹操 爬取成功！
第十二回·陶恭祖三让徐州    曹孟德大战吕布 爬取成功！
第十三回·李傕郭汜大交兵  杨奉董承双救驾 爬取成功！
第十四回·曹孟德移驾幸许都  吕奉先乘夜袭徐郡 爬取成功！
第十五回·太史慈酣斗小霸王  孙伯符大战严白虎 爬取成功！
第十六回·吕奉先射戟辕门    曹孟德败师淯水 爬取成功！
第十七回·袁公路大起七军    曹孟德会合三将 爬取成功！
第十八回·贾文和料敌决胜    夏侯惇拔矢啖睛 爬取成功！
第十九回·下邳城曹操鏖兵    白门楼吕布殒命 爬取成功！
第二十回·曹阿瞒许田打围    董国舅内阁受诏 爬取成功！
第二十一回·曹操煮酒论英雄  关公赚城斩车胄 爬取成功！
第二十二回·袁曹各起马步三军  关张共擒王刘二将 爬取成功！
第二十三回·祢正平裸衣骂贼    吉太医下毒遭刑 爬取成功！
第二十四回·国贼行凶杀贵妃    皇叔败走投袁绍 爬取成功！
第二十五回·屯土山关公约三事  救白马曹操解重围 爬取成功！
第二十六回·袁本初败兵折将    关云长挂印封金 爬取成功！
第二十七回·美髯公千里走单骑  汉寿侯五关斩六将 爬取成功！
第二十八回·斩蔡阳兄弟释疑    会古城主臣聚义 爬取成功！
第二十九回·小霸王怒斩于吉    碧眼儿坐领江东 爬取成功！
第三十回·战官渡本初败绩  劫乌巢孟德烧粮 爬取成功！
第三十一回·曹操仓亭破本初    玄德荆州依刘表 爬取成功！
第三十二回·夺冀州袁尚争锋    决漳河许攸献计 爬取成功！
第三十三回·曹丕乘乱纳甄氏    郭嘉遗计定辽东 爬取成功！
第三十四回·蔡夫人隔屏听密